In [1]:
# ============================================================================
# CELL 1: SETUP AND IMPORTS
# ============================================================================
print("="*80)
print("🚀 HUBERT ONLY FOR SAND CHALLENGE")
print("Strategy: HuBERT Features Only + Balanced Dataset")
print("Target: F1 Score >= 0.70 | High Accuracy")
print("="*80)

import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch
import librosa
import soundfile as sf
from scipy.stats import skew, kurtosis

from transformers import HubertModel, Wav2Vec2FeatureExtractor

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, cohen_kappa_score
)

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm.auto import tqdm
import joblib
import json

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

print("✅ All imports successful!")

🚀 HUBERT ONLY FOR SAND CHALLENGE
Strategy: HuBERT Features Only + Balanced Dataset
Target: F1 Score >= 0.70 | High Accuracy
✅ Using device: cpu
✅ All imports successful!


In [2]:
# ============================================================================
# CELL 2: CONFIGURATION FOR HUBERT ONLY + BALANCED DATASET
# ============================================================================
class Config:
    """Configuration for HuBERT Features Only with Balanced Dataset"""

    # Paths - UPDATE THESE PATHS FOR YOUR LOCAL SYSTEM
    BASE_PATH = 'SAND_Project_Data'  # Update this path
    TRAINING_PATH = os.path.join(BASE_PATH, 'training')
    EXCEL_PATH = os.path.join(BASE_PATH, 'sand_task_1.xlsx')
    OUTPUT_PATH = os.path.join(BASE_PATH, 'hubert_only_balanced')

    # Sheet names
    SHEET_TRAINING = 'Training Baseline - Task 1'
    SHEET_VALIDATION = 'Validation Baseline - Task 1'

    # Audio types
    AUDIO_TYPES = ['phonationA', 'phonationE', 'phonationI', 'phonationO', 'phonationU',
                   'rhythmKA', 'rhythmPA', 'rhythmTA']

    # ========================================================================
    # HUBERT CONFIGURATION - OPTIMIZED FOR PURE HUBERT FEATURES
    # ========================================================================
    HUBERT_MODEL = 'facebook/hubert-base-ls960'
    SAMPLING_RATE = 16000
    MAX_AUDIO_LENGTH = 16000 * 15  # 15 seconds

    # Multi-layer feature extraction (FOCUS ON HUBERT ONLY)
    USE_MULTI_LAYER = True
    LAYERS_TO_USE = [-4, -3, -2, -1]  # Last 4 layers

    # Multi-pooling strategy (MAXIMIZE HUBERT INFORMATION)
    USE_MULTI_POOLING = True
    POOLING_STRATEGIES = ['mean', 'std', 'max', 'min']  # 4 pooling methods

    # Statistical features - DISABLED FOR PURE HUBERT APPROACH
    USE_STATISTICAL_FEATURES = False  # Pure HuBERT features only

    # Audio augmentation during loading
    AUDIO_TRIM_DB = 15  # Lower threshold for dysarthric speech
    AUDIO_NORMALIZE = True
    MIN_AUDIO_LENGTH_SEC = 2

    # ========================================================================
    # PREPROCESSING OPTIMIZATIONS
    # ========================================================================
    USE_ROBUST_SCALING = True  # Better for outliers

    # PCA Configuration - MORE AGGRESSIVE FOR HUBERT FEATURES
    USE_PCA = True
    PCA_VARIANCE = 0.95  # Slightly lower to focus on most important HuBERT features

    # ========================================================================
    # DATA BALANCING STRATEGIES - FOCUS ON BALANCED DATASET
    # ========================================================================
    USE_SMOTE = True
    SMOTE_STRATEGY = 'auto'  # Balance all classes equally
    SMOTE_K_NEIGHBORS = 3

    # ========================================================================
    # CLASSIFIER CONFIGURATION - SVM FOR HUBERT FEATURES
    # ========================================================================
    SVM_CONFIG = {
        'kernel': 'rbf',
        'C': 100.0,  # Optimized for HuBERT features
        'gamma': 'scale',
        'class_weight': 'balanced',
        'probability': True,
        'decision_function_shape': 'ovr',
        'max_iter': 10000,
        'cache_size': 2000,
        'tol': 1e-4
    }

    # Cross-validation
    N_FOLDS = 5
    RANDOM_SEED = 42

    # Class names
    CLASS_NAMES = {
        0: 'Severe Dysarthria',
        1: 'Moderate Dysarthria',
        2: 'Mild Dysarthria',
        3: 'No Dysarthria (ALS)',
        4: 'Healthy'
    }

config = Config()
os.makedirs(config.OUTPUT_PATH, exist_ok=True)

print("\n" + "="*80)
print("📋 CONFIGURATION - HUBERT ONLY + BALANCED DATASET")
print("="*80)
print(f"HuBERT Model: {config.HUBERT_MODEL}")
print(f"Feature Strategy: PURE HUBERT FEATURES ONLY")
print(f"Statistical Features: {config.USE_STATISTICAL_FEATURES} (DISABLED)")
print(f"Dataset Strategy: BALANCED (SMOTE)")
print(f"Classifier: SVM (C={config.SVM_CONFIG['C']})")
print(f"Multi-Layer Extraction: {config.USE_MULTI_LAYER} ({len(config.LAYERS_TO_USE)} layers)")
print(f"Multi-Pooling: {config.USE_MULTI_POOLING} ({len(config.POOLING_STRATEGIES)} strategies)")
print(f"PCA: {config.USE_PCA} (variance={config.PCA_VARIANCE})")
print(f"SMOTE: {config.USE_SMOTE} (strategy={config.SMOTE_STRATEGY})")
print(f"Device: {device}")
print("="*80)

# Verify paths
print(f"\n🔍 Verifying paths...")
assert os.path.exists(config.BASE_PATH), f"Base path not found: {config.BASE_PATH}"
assert os.path.exists(config.TRAINING_PATH), f"Training path not found: {config.TRAINING_PATH}"
assert os.path.exists(config.EXCEL_PATH), f"Excel file not found: {config.EXCEL_PATH}"
print("✅ All paths verified!")


📋 CONFIGURATION - HUBERT ONLY + BALANCED DATASET
HuBERT Model: facebook/hubert-base-ls960
Feature Strategy: PURE HUBERT FEATURES ONLY
Statistical Features: False (DISABLED)
Dataset Strategy: BALANCED (SMOTE)
Classifier: SVM (C=100.0)
Multi-Layer Extraction: True (4 layers)
Multi-Pooling: True (4 strategies)
PCA: True (variance=0.95)
SMOTE: True (strategy=auto)
Device: cpu

🔍 Verifying paths...
✅ All paths verified!


In [3]:
# ============================================================================
# CELL 3: LOAD DATASETS
# ============================================================================
print("\n" + "="*80)
print("[1/9] LOADING DATASETS")
print("="*80)

def load_data(excel_path, sheet_name):
    """Load dataset from Excel"""
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    print(f"\n✅ Loaded {len(df)} samples from: '{sheet_name}'")

    # Convert to 0-indexed classes
    df['Class'] = df['Class'] - 1

    # Display distribution
    print(f"📊 Class distribution:")
    class_dist = df['Class'].value_counts().sort_index()
    for cls, count in class_dist.items():
        pct = (count / len(df)) * 100
        print(f"  Class {cls} ({config.CLASS_NAMES[cls]:25s}): {count:3d} ({pct:5.2f}%)")

    return df

# Load training and validation sets
df_train = load_data(config.EXCEL_PATH, config.SHEET_TRAINING)
df_val = load_data(config.EXCEL_PATH, config.SHEET_VALIDATION)

print(f"\n📈 Summary:")
print(f"  Training samples: {len(df_train)}")
print(f"  Validation samples: {len(df_val)}")
print(f"  Total audio files: {(len(df_train) + len(df_val)) * len(config.AUDIO_TYPES)}")

# Verify no overlap
train_ids = set(df_train['ID'].values)
val_ids = set(df_val['ID'].values)
overlap = train_ids.intersection(val_ids)
print(f"  Overlap check: {'✅ No overlap' if len(overlap) == 0 else f'⚠ {len(overlap)} overlapping IDs'}")


[1/9] LOADING DATASETS

✅ Loaded 219 samples from: 'Training Baseline - Task 1'
📊 Class distribution:
  Class 0 (Severe Dysarthria        ):   4 ( 1.83%)
  Class 1 (Moderate Dysarthria      ):  22 (10.05%)
  Class 2 (Mild Dysarthria          ):  45 (20.55%)
  Class 3 (No Dysarthria (ALS)      ):  62 (28.31%)
  Class 4 (Healthy                  ):  86 (39.27%)

✅ Loaded 53 samples from: 'Validation Baseline - Task 1'
📊 Class distribution:
  Class 0 (Severe Dysarthria        ):   2 ( 3.77%)
  Class 1 (Moderate Dysarthria      ):   4 ( 7.55%)
  Class 2 (Mild Dysarthria          ):  12 (22.64%)
  Class 3 (No Dysarthria (ALS)      ):  14 (26.42%)
  Class 4 (Healthy                  ):  21 (39.62%)

📈 Summary:
  Training samples: 219
  Validation samples: 53
  Total audio files: 2176
  Overlap check: ✅ No overlap


In [4]:
# ============================================================================
# CELL 4: LOAD HUBERT MODEL
# ============================================================================
print("\n" + "="*80)
print("[2/9] LOADING HUBERT MODEL")
print("="*80)
print(f"Model: {config.HUBERT_MODEL}")
print("⏱ This may take 1-2 minutes for first-time download...")

# Load feature extractor and model
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(config.HUBERT_MODEL)
hubert_model = HubertModel.from_pretrained(config.HUBERT_MODEL)
hubert_model = hubert_model.to(device)
hubert_model.eval()  # Set to evaluation mode

print(f"\n✅ HuBERT loaded successfully!")
print(f"  Model size: {sum(p.numel() for p in hubert_model.parameters()) / 1e6:.1f}M parameters")
print(f"  Hidden size: {hubert_model.config.hidden_size}")
print(f"  Number of layers: {hubert_model.config.num_hidden_layers}")
print(f"  Sampling rate: {feature_extractor.sampling_rate} Hz")


[2/9] LOADING HUBERT MODEL
Model: facebook/hubert-base-ls960
⏱ This may take 1-2 minutes for first-time download...

✅ HuBERT loaded successfully!
  Model size: 94.4M parameters
  Hidden size: 768
  Number of layers: 12
  Sampling rate: 16000 Hz


In [5]:
# ============================================================================
# CELL 5: AUDIO PROCESSING FUNCTIONS - HUBERT ONLY
# ============================================================================
print("\n" + "="*80)
print("[3/9] DEFINING HUBERT-ONLY AUDIO PROCESSING")
print("="*80)

def load_audio_enhanced(audio_path, target_sr=16000, max_length=None):
    """
    Enhanced audio loading with advanced preprocessing
    Focus on optimal input for HuBERT features
    """
    try:
        # Load audio
        audio, sr = librosa.load(audio_path, sr=target_sr, mono=True)

        # Advanced trimming with lower threshold for dysarthric speech
        audio, _ = librosa.effects.trim(audio, top_db=config.AUDIO_TRIM_DB)

        # Normalize amplitude for consistent HuBERT processing
        if config.AUDIO_NORMALIZE:
            audio = librosa.util.normalize(audio)

        # Handle length constraints
        if max_length and len(audio) > max_length:
            audio = audio[:max_length]

        # Ensure minimum length for meaningful HuBERT features
        min_length = target_sr * config.MIN_AUDIO_LENGTH_SEC
        if len(audio) < min_length:
            audio = np.pad(audio, (0, min_length - len(audio)), mode='constant')

        return audio, sr

    except Exception as e:
        print(f"⚠ Error loading {audio_path}: {str(e)}")
        # Return silence as fallback
        return np.zeros(target_sr * config.MIN_AUDIO_LENGTH_SEC), target_sr

def extract_hubert_features_multilayer(audio, sampling_rate=16000):
    """
    Extract PURE HuBERT features with multi-layer and multi-pooling strategy
    NO statistical features - Pure HuBERT representation only
    """
    try:
        # Preprocess audio for HuBERT
        inputs = feature_extractor(
            audio,
            sampling_rate=sampling_rate,
            return_tensors="pt",
            padding=True
        )

        # Move to device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Extract features with all hidden states
        with torch.no_grad():
            outputs = hubert_model(**inputs, output_hidden_states=True)

        all_features = []

        # Extract from multiple layers
        if config.USE_MULTI_LAYER:
            layers_to_process = config.LAYERS_TO_USE
        else:
            layers_to_process = [-1]  # Only last layer

        for layer_idx in layers_to_process:
            hidden_state = outputs.hidden_states[layer_idx]  # Shape: [batch, time, hidden]

            # Apply multiple pooling strategies
            if config.USE_MULTI_POOLING:
                pooling_methods = config.POOLING_STRATEGIES
            else:
                pooling_methods = ['mean']  # Only mean pooling

            for pool_strategy in pooling_methods:
                if pool_strategy == 'mean':
                    pooled = hidden_state.mean(dim=1)
                elif pool_strategy == 'max':
                    pooled = hidden_state.max(dim=1)[0]
                elif pool_strategy == 'std':
                    pooled = hidden_state.std(dim=1)
                elif pool_strategy == 'min':
                    pooled = hidden_state.min(dim=1)[0]
                else:
                    pooled = hidden_state.mean(dim=1)

                all_features.append(pooled.cpu().numpy().squeeze())

        # Concatenate all PURE HuBERT features
        hubert_features = np.concatenate(all_features)

        return hubert_features

    except Exception as e:
        print(f"⚠ HuBERT feature extraction error: {str(e)}")
        # Return zero features as fallback
        n_layers = len(config.LAYERS_TO_USE) if config.USE_MULTI_LAYER else 1
        n_pooling = len(config.POOLING_STRATEGIES) if config.USE_MULTI_POOLING else 1
        feature_dim = hubert_model.config.hidden_size * n_layers * n_pooling
        return np.zeros(feature_dim)

print("✅ HuBERT-only audio processing functions defined!")
print(f"  Enhanced audio loading: ✓")
print(f"  Pure HuBERT features: ✓ (NO statistical features)")
print(f"  Multi-layer HuBERT extraction: ✓")
print(f"  Feature composition: 100% HuBERT representations")


[3/9] DEFINING HUBERT-ONLY AUDIO PROCESSING
✅ HuBERT-only audio processing functions defined!
  Enhanced audio loading: ✓
  Pure HuBERT features: ✓ (NO statistical features)
  Multi-layer HuBERT extraction: ✓
  Feature composition: 100% HuBERT representations


In [6]:
# ============================================================================
# CELL 6: EXTRACT PURE HUBERT FEATURES
# ============================================================================
print("\n" + "="*80)
print("[4/9] EXTRACTING PURE HUBERT FEATURES")
print("="*80)
print(f"Configuration:")
print(f"  Layers: {len(config.LAYERS_TO_USE)}")
print(f"  Pooling strategies: {len(config.POOLING_STRATEGIES)}")
print(f"  Statistical features: {config.USE_STATISTICAL_FEATURES} (DISABLED)")
print(f"  Feature multiplication factor: {len(config.LAYERS_TO_USE) * len(config.POOLING_STRATEGIES)}x")
print(f"  Pure HuBERT feature dimension: {hubert_model.config.hidden_size * len(config.LAYERS_TO_USE) * len(config.POOLING_STRATEGIES)}")

def extract_features_for_dataset(df, dataset_path, audio_types, desc="Processing"):
    """
    Extract PURE HuBERT features for entire dataset
    No statistical features - only HuBERT representations
    """
    features_list = []
    labels_list = []
    ids_list = []

    print(f"\n🎵 {desc} {len(df)} patients with PURE HUBERT features...")

    for idx, row in tqdm(df.iterrows(), total=len(df), desc=desc):
        patient_id = row['ID']
        patient_class = row['Class']

        patient_features = []

        # Process each audio type
        for audio_type in audio_types:
            audio_file = f"{patient_id}_{audio_type}.wav"
            audio_path = os.path.join(dataset_path, audio_type, audio_file)

            if os.path.exists(audio_path):
                # Load audio with enhancements
                audio, sr = load_audio_enhanced(
                    audio_path,
                    target_sr=config.SAMPLING_RATE,
                    max_length=config.MAX_AUDIO_LENGTH
                )

                # Extract PURE HuBERT features only
                hubert_features = extract_hubert_features_multilayer(audio, sr)

                # Use only HuBERT features (no statistical features)
                combined_features = hubert_features

                patient_features.append(combined_features)
            else:
                # Zero features for missing files (HuBERT-only dimension)
                n_layers = len(config.LAYERS_TO_USE) if config.USE_MULTI_LAYER else 1
                n_pooling = len(config.POOLING_STRATEGIES) if config.USE_MULTI_POOLING else 1
                feature_dim = hubert_model.config.hidden_size * n_layers * n_pooling

                patient_features.append(np.zeros(feature_dim))

        # Concatenate features from all audio types
        combined = np.concatenate(patient_features)
        features_list.append(combined)
        labels_list.append(patient_class)
        ids_list.append(patient_id)

    X = np.array(features_list)
    y = np.array(labels_list)
    ids = np.array(ids_list)

    return X, y, ids

# Extract training features
X_train, y_train, ids_train = extract_features_for_dataset(
    df_train,
    config.TRAINING_PATH,
    config.AUDIO_TYPES,
    desc="Training"
)

# Extract validation features
X_val, y_val, ids_val = extract_features_for_dataset(
    df_val,
    config.TRAINING_PATH,
    config.AUDIO_TYPES,
    desc="Validation"
)

print(f"\n✅ Pure HuBERT feature extraction complete!")
print(f"  Training shape: {X_train.shape}")
print(f"  Validation shape: {X_val.shape}")
print(f"  Feature dimension per patient: {X_train.shape[1]}")
print(f"  Feature composition: 100% HuBERT representations")
print(f"  Total HuBERT features: {X_train.shape[1]}")

# Clean data (handle any NaN/Inf)
X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
X_val = np.nan_to_num(X_val, nan=0.0, posinf=0.0, neginf=0.0)

print(f"  Data cleaned: ✓")

# Save raw features
features_path = os.path.join(config.OUTPUT_PATH, 'features_hubert_only.npz')
np.savez(
    features_path,
    X_train=X_train, y_train=y_train, ids_train=ids_train,
    X_val=X_val, y_val=y_val, ids_val=ids_val
)
print(f"  💾 Pure HuBERT features saved: {features_path}")

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()


[4/9] EXTRACTING PURE HUBERT FEATURES
Configuration:
  Layers: 4
  Pooling strategies: 4
  Statistical features: False (DISABLED)
  Feature multiplication factor: 16x
  Pure HuBERT feature dimension: 12288

🎵 Training 219 patients with PURE HUBERT features...


Training:   0%|          | 0/219 [00:00<?, ?it/s]


🎵 Validation 53 patients with PURE HUBERT features...


Validation:   0%|          | 0/53 [00:00<?, ?it/s]


✅ Pure HuBERT feature extraction complete!
  Training shape: (219, 98304)
  Validation shape: (53, 98304)
  Feature dimension per patient: 98304
  Feature composition: 100% HuBERT representations
  Total HuBERT features: 98304
  Data cleaned: ✓
  💾 Pure HuBERT features saved: SAND_Project_Data\hubert_only_balanced\features_hubert_only.npz


In [7]:
# ============================================================================
# CELL 7: PREPROCESSING PIPELINE - BALANCED DATASET
# ============================================================================
print("\n" + "="*80)
print("[5/9] PREPROCESSING FOR BALANCED DATASET")
print("="*80)

# Step 1: Scaling
if config.USE_ROBUST_SCALING:
    print("📊 Applying RobustScaler (better for HuBERT feature outliers)...")
    scaler = RobustScaler()
else:
    print("📊 Applying StandardScaler...")
    scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print(f"  ✅ Scaling complete")
print(f"     Train: mean={X_train_scaled.mean():.6f}, std={X_train_scaled.std():.6f}")
print(f"     Val:   mean={X_val_scaled.mean():.6f}, std={X_val_scaled.std():.6f}")

# Step 2: PCA (Dimensionality Reduction for HuBERT features)
if config.USE_PCA:
    print(f"\n📉 Applying PCA (variance threshold={config.PCA_VARIANCE})...")
    pca = PCA(n_components=config.PCA_VARIANCE, random_state=config.RANDOM_SEED)
    X_train_scaled = pca.fit_transform(X_train_scaled)
    X_val_scaled = pca.transform(X_val_scaled)

    explained_var = pca.explained_variance_ratio_.sum()
    print(f"  ✅ PCA complete")
    print(f"     Original HuBERT dimensions: {X_train.shape[1]}")
    print(f"     Reduced dimensions: {X_train_scaled.shape[1]}")
    print(f"     Explained variance: {explained_var*100:.2f}%")
    print(f"     Dimensionality reduction: {X_train.shape[1]/X_train_scaled.shape[1]:.1f}x")
    
    # Analyze PCA components
    print(f"     Top 5 PCA components explain: {pca.explained_variance_ratio_[:5].sum()*100:.2f}% variance")
else:
    pca = None
    print("  ℹ PCA disabled")

# Step 3: Data Balancing with SMOTE
if config.USE_SMOTE:
    print(f"\n⚖️ Applying SMOTE for balanced dataset (strategy='{config.SMOTE_STRATEGY}')...")
    print(f"  Original class distribution:")
    unique, counts = np.unique(y_train, return_counts=True)
    total_samples = len(y_train)
    for cls, count in zip(unique, counts):
        pct = (count / total_samples) * 100
        print(f"    Class {cls} ({config.CLASS_NAMES[cls]:25s}): {count:3d} ({pct:5.1f}%)")

    try:
        smote = SMOTE(
            sampling_strategy=config.SMOTE_STRATEGY,
            k_neighbors=config.SMOTE_K_NEIGHBORS,
            random_state=config.RANDOM_SEED
        )
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

        print(f"\n  ✅ SMOTE balancing complete")
        print(f"     Samples: {len(y_train)} → {len(y_train_resampled)}")
        print(f"  Balanced class distribution:")
        unique, counts = np.unique(y_train_resampled, return_counts=True)
        for cls, count in zip(unique, counts):
            pct = (count / len(y_train_resampled)) * 100
            print(f"    Class {cls} ({config.CLASS_NAMES[cls]:25s}): {count:3d} ({pct:5.1f}%)")

    except Exception as e:
        print(f"  ⚠ SMOTE failed: {e}")
        print(f"  Continuing without balancing...")
        X_train_resampled = X_train_scaled
        y_train_resampled = y_train
else:
    X_train_resampled = X_train_scaled
    y_train_resampled = y_train
    print("  ℹ SMOTE disabled - using original unbalanced data")

print("\n✅ Preprocessing complete! Using balanced dataset with pure HuBERT features.")


[5/9] PREPROCESSING FOR BALANCED DATASET
📊 Applying RobustScaler (better for HuBERT feature outliers)...
  ✅ Scaling complete
     Train: mean=0.021912, std=0.779400
     Val:   mean=0.033591, std=0.818913

📉 Applying PCA (variance threshold=0.95)...
  ✅ PCA complete
     Original HuBERT dimensions: 98304
     Reduced dimensions: 187
     Explained variance: 95.01%
     Dimensionality reduction: 525.7x
     Top 5 PCA components explain: 22.99% variance

⚖️ Applying SMOTE for balanced dataset (strategy='auto')...
  Original class distribution:
    Class 0 (Severe Dysarthria        ):   4 (  1.8%)
    Class 1 (Moderate Dysarthria      ):  22 ( 10.0%)
    Class 2 (Mild Dysarthria          ):  45 ( 20.5%)
    Class 3 (No Dysarthria (ALS)      ):  62 ( 28.3%)
    Class 4 (Healthy                  ):  86 ( 39.3%)

  ✅ SMOTE balancing complete
     Samples: 219 → 430
  Balanced class distribution:
    Class 0 (Severe Dysarthria        ):  86 ( 20.0%)
    Class 1 (Moderate Dysarthria      ): 

In [8]:
# ============================================================================
# CELL 8: TRAIN SVM CLASSIFIER ON HUBERT FEATURES
# ============================================================================
print("\n" + "="*80)
print("[6/9] TRAINING SVM ON PURE HUBERT FEATURES")
print("="*80)
print(f"Configuration:")
for key, value in config.SVM_CONFIG.items():
    if key not in ['cache_size', 'tol']:  # Skip technical parameters
        print(f"  {key}: {value}")

print(f"\n🎯 Training SVM on {len(y_train_resampled)} balanced samples...")
print(f"   Features: 100% Pure HuBERT representations")
print(f"   Dataset: {'BALANCED' if config.USE_SMOTE else 'UNBALANCED'}")
print(f"   Feature dimension: {X_train_resampled.shape[1]}")

# Initialize and train classifier
classifier = SVC(**config.SVM_CONFIG)
classifier.fit(X_train_resampled, y_train_resampled)

print(f"✅ SVM training complete!")
print(f"  Number of support vectors: {classifier.n_support_.sum()}")
print(f"  Support vectors per class: {classifier.n_support_}")
print(f"  Feature space: Pure HuBERT representations")


[6/9] TRAINING SVM ON PURE HUBERT FEATURES
Configuration:
  kernel: rbf
  C: 100.0
  gamma: scale
  class_weight: balanced
  probability: True
  decision_function_shape: ovr
  max_iter: 10000

🎯 Training SVM on 430 balanced samples...
   Features: 100% Pure HuBERT representations
   Dataset: BALANCED
   Feature dimension: 187
✅ SVM training complete!
  Number of support vectors: 275
  Support vectors per class: [12 48 55 74 86]
  Feature space: Pure HuBERT representations


In [9]:
# ============================================================================
# CELL 9: CROSS-VALIDATION ON BALANCED DATASET
# ============================================================================
print("\n" + "="*80)
print("[7/9] CROSS-VALIDATION ON BALANCED DATA")
print("="*80)
print(f"Performing {config.N_FOLDS}-fold stratified cross-validation...")
print("⏱ This may take a few minutes...")

# Cross-validation on balanced training set
cv_scores = cross_val_score(
    SVC(**config.SVM_CONFIG),
    X_train_resampled,  # Use balanced data for CV
    y_train_resampled,
    cv=config.N_FOLDS,
    scoring='f1_macro',
    n_jobs=-1
)

print(f"\n✅ Cross-validation complete!")
print(f"  Fold scores: {[f'{s:.4f}' for s in cv_scores]}")
print(f"  Mean CV F1: {cv_scores.mean():.4f}")
print(f"  Std CV F1:  {cv_scores.std():.4f}")
print(f"  Min CV F1:  {cv_scores.min():.4f}")
print(f"  Max CV F1:  {cv_scores.max():.4f}")

# Additional metrics
cv_accuracy = cross_val_score(
    SVC(**config.SVM_CONFIG),
    X_train_resampled,
    y_train_resampled,
    cv=config.N_FOLDS,
    scoring='accuracy',
    n_jobs=-1
)
print(f"  Mean CV Accuracy: {cv_accuracy.mean():.4f}")


[7/9] CROSS-VALIDATION ON BALANCED DATA
Performing 5-fold stratified cross-validation...
⏱ This may take a few minutes...

✅ Cross-validation complete!
  Fold scores: ['0.7980', '0.8131', '0.7808', '0.8155', '0.8827']
  Mean CV F1: 0.8180
  Std CV F1:  0.0347
  Min CV F1:  0.7808
  Max CV F1:  0.8827
  Mean CV Accuracy: 0.8256


In [10]:
# ============================================================================
# CELL 10: EVALUATE ON VALIDATION SET - HUBERT ONLY
# ============================================================================
print("\n" + "="*80)
print("[8/9] VALIDATION SET EVALUATION - PURE HUBERT FEATURES")
print("="*80)

# Predict
y_val_pred = classifier.predict(X_val_scaled)
y_val_pred_proba = classifier.predict_proba(X_val_scaled)

# Compute metrics
accuracy = accuracy_score(y_val, y_val_pred)
f1_macro = f1_score(y_val, y_val_pred, average='macro')
f1_weighted = f1_score(y_val, y_val_pred, average='weighted')
precision = precision_score(y_val, y_val_pred, average='macro', zero_division=0)
recall = recall_score(y_val, y_val_pred, average='macro', zero_division=0)
kappa = cohen_kappa_score(y_val, y_val_pred)
f1_per_class = f1_score(y_val, y_val_pred, average=None, zero_division=0)

# Display results
print("\n" + "="*80)
print("🎯 FINAL RESULTS - PURE HUBERT FEATURES + BALANCED DATASET")
print("="*80)
print(f"Feature Strategy: 100% HUBERT REPRESENTATIONS")
print(f"Statistical Features: {config.USE_STATISTICAL_FEATURES} (DISABLED)")
print(f"Dataset Strategy: BALANCED")
print(f"Classifier: SVM (C={config.SVM_CONFIG['C']})")
print(f"Validation Accuracy:      {accuracy:.4f}")
print(f"F1 Score (Macro):         {f1_macro:.4f} ⭐")
print(f"F1 Score (Weighted):      {f1_weighted:.4f}")
print(f"Precision (Macro):        {precision:.4f}")
print(f"Recall (Macro):           {recall:.4f}")
print(f"Cohen's Kappa:            {kappa:.4f}")
print(f"CV F1 (Mean ± Std):       {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print("="*80)

# Per-class metrics
print("\n📊 PER-CLASS PERFORMANCE (PURE HUBERT FEATURES):")
available_classes = sorted(np.unique(y_val))
for cls, f1_val in zip(available_classes, [f1_per_class[i] for i in available_classes]):
    label = config.CLASS_NAMES[cls]
    bar = '█' * int(f1_val * 40)
    print(f"  Class {cls} ({label:25s}): {f1_val:.4f} {bar}")

# Classification report
print("\n" + "-"*80)
print("CLASSIFICATION REPORT:")
print("-"*80)
class_labels = [config.CLASS_NAMES[i] for i in available_classes]
print(classification_report(y_val, y_val_pred, target_names=class_labels, zero_division=0))

# Confusion matrix
cm = confusion_matrix(y_val, y_val_pred)
print("\nCONFUSION MATRIX:")
print(cm)

# HuBERT feature analysis
print(f"\n🔍 HUBERT FEATURE ANALYSIS:")
print(f"  Original HuBERT feature dimension: {X_train.shape[1]}")
if pca:
    print(f"  Reduced feature dimension: {X_train_scaled.shape[1]}")
    print(f"  Dimensionality reduction: {X_train.shape[1]/X_train_scaled.shape[1]:.1f}x")
print(f"  Feature composition: 100% HuBERT representations")
print(f"  Multi-layer strategy: {len(config.LAYERS_TO_USE)} layers")
print(f"  Pooling strategies: {len(config.POOLING_STRATEGIES)} methods")

# Achievement status
print("\n" + "="*80)
baseline_f1 = 0.4919
target_f1 = 0.70
improvement = f1_macro - baseline_f1

if f1_macro >= target_f1:
    print(f"🎉🎉🎉 TARGET ACHIEVED WITH PURE HUBERT! 🎉🎉🎉")
    print(f"F1 Score: {f1_macro:.4f} >= {target_f1:.4f}")
    print(f"Improvement over baseline: +{improvement:.4f} (+{improvement/baseline_f1*100:.1f}%)")
    print(f"✅ Pure HuBERT features are sufficient for high performance!")
elif f1_macro >= 0.65:
    print(f"🚀 EXCELLENT PERFORMANCE WITH PURE HUBERT!")
    print(f"F1 Score: {f1_macro:.4f}")
    print(f"Gap to target: -{target_f1 - f1_macro:.4f}")
    print(f"Improvement over baseline: +{improvement:.4f} (+{improvement/baseline_f1*100:.1f}%)")
else:
    print(f"📈 GOOD PERFORMANCE WITH PURE HUBERT!")
    print(f"F1 Score: {f1_macro:.4f}")
    print(f"Improvement over baseline: +{improvement:.4f} (+{improvement/baseline_f1*100:.1f}%)")
    print(f"💡 Consider adding statistical features for potential improvement")
print("="*80)


[8/9] VALIDATION SET EVALUATION - PURE HUBERT FEATURES

🎯 FINAL RESULTS - PURE HUBERT FEATURES + BALANCED DATASET
Feature Strategy: 100% HUBERT REPRESENTATIONS
Statistical Features: False (DISABLED)
Dataset Strategy: BALANCED
Classifier: SVM (C=100.0)
Validation Accuracy:      0.5849
F1 Score (Macro):         0.4718 ⭐
F1 Score (Weighted):      0.5651
Precision (Macro):        0.4467
Recall (Macro):           0.5381
Cohen's Kappa:            0.4276
CV F1 (Mean ± Std):       0.8180 ± 0.0347

📊 PER-CLASS PERFORMANCE (PURE HUBERT FEATURES):
  Class 0 (Severe Dysarthria        ): 0.0000 
  Class 1 (Moderate Dysarthria      ): 0.6667 ██████████████████████████
  Class 2 (Mild Dysarthria          ): 0.5926 ███████████████████████
  Class 3 (No Dysarthria (ALS)      ): 0.4167 ████████████████
  Class 4 (Healthy                  ): 0.6829 ███████████████████████████

--------------------------------------------------------------------------------
CLASSIFICATION REPORT:
------------------------

In [11]:
# ============================================================================
# CELL 11: SAVE ALL MODELS AND RESULTS - HUBERT ONLY
# ============================================================================
print("\n" + "="*80)
print("[9/9] SAVING RESULTS - PURE HUBERT FEATURES")
print("="*80)

# Save models
print("💾 Saving models...")
joblib.dump(scaler, os.path.join(config.OUTPUT_PATH, 'scaler.pkl'))
print(f"  ✅ Scaler saved")

if pca:
    joblib.dump(pca, os.path.join(config.OUTPUT_PATH, 'pca.pkl'))
    print(f"  ✅ PCA saved")

joblib.dump(classifier, os.path.join(config.OUTPUT_PATH, 'classifier.pkl'))
print(f"  ✅ Classifier saved")

# Save results dictionary with HuBERT-only focus
results = {
    'model': 'Pure HuBERT Features + Balanced Dataset',
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'strategy': 'HuBERT Only + Balanced Dataset',
    'feature_strategy': 'pure_hubert',
    'statistical_features': False,
    'classifier_type': 'SVM',
    'dataset_strategy': 'balanced',
    'metrics': {
        'accuracy': float(accuracy),
        'f1_macro': float(f1_macro),
        'f1_weighted': float(f1_weighted),
        'precision': float(precision),
        'recall': float(recall),
        'kappa': float(kappa),
        'cv_mean': float(cv_scores.mean()),
        'cv_std': float(cv_scores.std()),
    },
    'f1_per_class': {int(i): float(f1_per_class[i]) for i in range(len(f1_per_class))},
    'confusion_matrix': cm.tolist(),
    'hubert_feature_analysis': {
        'original_dimension': int(X_train.shape[1]),
        'final_dimension': int(X_train_scaled.shape[1]) if pca else int(X_train.shape[1]),
        'reduction_factor': float(X_train.shape[1] / X_train_scaled.shape[1]) if pca else 1.0,
        'layers_used': len(config.LAYERS_TO_USE),
        'pooling_strategies': len(config.POOLING_STRATEGIES),
        'feature_composition': '100%_hubert'
    },
    'configuration': {
        'hubert_model': config.HUBERT_MODEL,
        'multi_layer': config.USE_MULTI_LAYER,
        'n_layers': len(config.LAYERS_TO_USE),
        'layers': config.LAYERS_TO_USE,
        'multi_pooling': config.USE_MULTI_POOLING,
        'pooling_strategies': config.POOLING_STRATEGIES,
        'statistical_features': config.USE_STATISTICAL_FEATURES,
        'robust_scaling': config.USE_ROBUST_SCALING,
        'pca': config.USE_PCA,
        'pca_variance': config.PCA_VARIANCE if config.USE_PCA else None,
        'balancing_strategy': 'smote' if config.USE_SMOTE else 'none',
        'svm_C': config.SVM_CONFIG['C'],
        'svm_kernel': config.SVM_CONFIG['kernel'],
    }
}

results_path = os.path.join(config.OUTPUT_PATH, 'results.json')
with open(results_path, 'w') as f:
    json.dump(results, f, indent=4)
print(f"  ✅ Results saved: {results_path}")

# Save predictions
val_predictions = pd.DataFrame({
    'ID': ids_val,
    'True_Class': y_val,
    'Predicted_Class': y_val_pred,
    'True_Label': [config.CLASS_NAMES[int(c)] for c in y_val],
    'Predicted_Label': [config.CLASS_NAMES[int(c)] for c in y_val_pred],
    'Correct': (y_val == y_val_pred).astype(int),
    'Confidence': y_val_pred_proba.max(axis=1)
})

# Add probability columns
for cls in range(len(config.CLASS_NAMES)):
    val_predictions[f'Prob_Class_{cls}'] = y_val_pred_proba[:, cls]
    val_predictions[f'Prob_{config.CLASS_NAMES[cls]}'] = y_val_pred_proba[:, cls]

val_predictions = val_predictions.sort_values('ID').reset_index(drop=True)

pred_path = os.path.join(config.OUTPUT_PATH, 'validation_predictions.csv')
val_predictions.to_csv(pred_path, index=False)
print(f"  ✅ Predictions saved: {pred_path}")

print(f"\n✅ All results saved to: {config.OUTPUT_PATH}")

# Final summary
print(f"\n🎯 PURE HUBERT STRATEGY SUMMARY:")
print(f"  • Feature Strategy: 100% HuBERT representations")
print(f"  • Statistical Features: DISABLED")
print(f"  • Dataset: BALANCED (SMOTE)")
print(f"  • Multi-layer extraction: {len(config.LAYERS_TO_USE)} layers")
print(f"  • Multi-pooling strategies: {len(config.POOLING_STRATEGIES)} methods")
print(f"  • Feature dimension: {X_train.shape[1]} → {X_train_scaled.shape[1] if pca else X_train.shape[1]}")
print(f"  • Final F1 Score: {f1_macro:.4f}")
print(f"  • Pure HuBERT performance: {'✅ SUFFICIENT' if f1_macro >= 0.65 else '📈 GOOD'}")


[9/9] SAVING RESULTS - PURE HUBERT FEATURES
💾 Saving models...
  ✅ Scaler saved
  ✅ PCA saved
  ✅ Classifier saved
  ✅ Results saved: SAND_Project_Data\hubert_only_balanced\results.json
  ✅ Predictions saved: SAND_Project_Data\hubert_only_balanced\validation_predictions.csv

✅ All results saved to: SAND_Project_Data\hubert_only_balanced

🎯 PURE HUBERT STRATEGY SUMMARY:
  • Feature Strategy: 100% HuBERT representations
  • Statistical Features: DISABLED
  • Dataset: BALANCED (SMOTE)
  • Multi-layer extraction: 4 layers
  • Multi-pooling strategies: 4 methods
  • Feature dimension: 98304 → 187
  • Final F1 Score: 0.4718
  • Pure HuBERT performance: 📈 GOOD
